# Node-breaker and bus-branch topology

In PowSyBl, there are two possible voltage level topologies:

- **Node-breaker topology**: This topology describes all the internal connections of a voltage level, including the different switching stages between an element connected to the voltage level and one or more of its busbars. It is the most complete description of the internal connections.

- **Bus-breaker topology**: It is a simpler description of a voltage level where we can 


A voltage level can be described in node-breaker topology by firstly defining the voltage level object itself:




In [1]:
import pypowsybl as pp

grid = pp.network.create_empty()

grid.create_substations(id='SSID1', name='SS1') # For the purpose of this tutorial, we use descriptive IDs name, but we encourage to use randomly generated IDs, such as UUIDs.
grid.create_voltage_levels(id='VLID1', name='VL1', substation_id='SSID1', nominal_v=400, high_voltage_limit=420, low_voltage_limit=380, topology_kind='NODE_BREAKER')

Right now, we can check its topology, although it is of course empty.

In [2]:
grid.get_node_breaker_topology(voltage_level_id='VLID1').nodes


,connectable_id,connectable_type
node,,


In [3]:
grid.get_node_breaker_topology(voltage_level_id='VLID1').switches


,name,kind,open,retained,node1,node2
id,,,,,,


We can start off by adding busbars, shunt elements as well as the switch connections from element to busbar.

In [4]:
grid.create_busbar_sections(id='BBID1', name='BB1', voltage_level_id='VLID1', node=0)
grid.create_busbar_sections(id='BBID2', name='BB2', voltage_level_id='VLID1', node=1)
grid.create_generators(id='GENID1', name='GEN1', voltage_level_id='VLID1', node=2, min_p=0.0, max_p=100.0, target_p=80.0, target_q=20.0, voltage_regulator_on=False)
grid.create_loads(id='LOADID1', name='LOAD1', voltage_level_id='VLID1', node=3, p0=5.0, q0=2.0)
grid.create_switches(id='SWID1', name='LOAD1_SA_BB1', kind='DISCONNECTOR', voltage_level_id='VLID1', node1=0, node2=3, open=False)
grid.create_switches(id='SWID2', name='GEN1_SA_BB2', kind='DISCONNECTOR', voltage_level_id='VLID1', node1=1, node2=2, open=False)
grid.create_switches(id='SWID3', name='BB1_BB2', kind='DISCONNECTOR', voltage_level_id='VLID1', node1=0, node2=1, open=False)

We can check now again the elements connected to each internal node of this voltage level.

In [5]:
grid.get_node_breaker_topology(voltage_level_id='VLID1').nodes


,connectable_id,connectable_type
node,,
0,BBID1,BUSBAR_SECTION
1,BBID2,BUSBAR_SECTION
2,GENID1,GENERATOR
3,LOADID1,LOAD


We can also check the switch configuration.

In [6]:
grid.get_node_breaker_topology(voltage_level_id='VLID1').switches


,name,kind,open,retained,node1,node2
id,,,,,,
SWID1,LOAD1_SA_BB1,DISCONNECTOR,False,False,0,3
SWID2,GEN1_SA_BB2,DISCONNECTOR,False,False,1,2
SWID3,BB1_BB2,DISCONNECTOR,False,False,0,1


Although we have not defined any bus in this network, we can call the following function to check what are the buses obtained after the automatic topology processing that PowSyBl does.

In [14]:
grid.get_busbar_sections()

,name,v,angle,voltage_level_id,bus_id,connected
id,,,,,,


We can also see the resulting bus-breaker topology of this voltage level, even though it is still in node-breaker topology, aswell as the resulting element to bus connections.

In [8]:
grid.get_bus_breaker_topology(voltage_level_id='VLID1').elements

,type,bus_id,side
id,,,
BBID1,BUSBAR_SECTION,VLID1_0,
BBID2,BUSBAR_SECTION,VLID1_0,
GENID1,GENERATOR,VLID1_0,
LOADID1,LOAD,VLID1_0,


Now, we can transform the voltage level into a bus-breaker topology. Doing so, we will remove all the internal connections and switches that are not retained, yielding a simplified topology, and the deleted information will not be available anymore.

In [9]:
grid.update_voltage_levels(id='VLID1', topology_kind = 'BUS_BREAKER')

In [10]:
grid.get_voltage_levels(all_attributes=True)

,name,substation_id,nominal_v,high_voltage_limit,low_voltage_limit,fictitious,topology_kind
id,,,,,,,
VLID1,VL1,SSID1,400.0,420.0,380.0,False,BUS_BREAKER


In [11]:
grid.get_bus_breaker_topology(voltage_level_id='VLID1').buses

,name,bus_id
id,,
VLID1_0,VL1_0,


In [12]:
diag = grid.get_single_line_diagram('VLID1')
diag

If we try to get the node breaker topology, we will get a PyPowSyBl error telling us that this type of configuration does not support this call.

In [35]:
grid.get_node_breaker_topology(voltage_level_id='VLID1')

PyPowsyblError: Not supported in a bus breaker topology

In [29]:
grid.get_buses()


,name,v_mag,v_angle,connected_component,synchronous_component,voltage_level_id
id,,,,,,


In [32]:
grid.get_bus_breaker_topology(voltage_level_id='VLID1').elements

,type,bus_id,side
id,,,
LOADID1,LOAD,,
GENID1,GENERATOR,,
